Opensmile Audio Feature

In [ ]:
import opensmile
import os
import pandas as pd

path_to_directory = 'yue_audio'
mp3_files = [os.path.join(path_to_directory, f) for f in os.listdir(path_to_directory) if f.endswith('.wav')]
smile = opensmile.Smile(feature_set=opensmile.FeatureSet.ComParE_2016, feature_level=opensmile.FeatureLevel.Functionals)
combined_df = pd.DataFrame()

for mp3_file in mp3_files:
    y = smile.process_file(mp3_file)
    combined_df = pd.concat([combined_df, y], ignore_index=True)

combined_df.to_csv('yue_audio/yue_audio_feature.csv', index=False)

Stable Diffusion Visual Feature

In [ ]:
from diffusers import StableDiffusionPipeline
import torch
import pickle

device = 'cuda:1'

pipe = StableDiffusionPipeline.from_pretrained("text2img_model/taiyi",requires_safety_checker=False).to(device)

def get_unet_output(prompt: str, timestep: int = 999) -> torch.Tensor:

    text_inputs = pipe.tokenizer(
        prompt,
        padding="max_length",
        max_length=16,
        truncation=True,
        return_tensors="pt",
    )
    text_input_ids = text_inputs.input_ids.to(device)
    text_embeddings = pipe.text_encoder(text_input_ids)[0]
    
    latents = torch.randn(
        (1, pipe.unet.config.in_channels, 64, 64),  # 64x64 对应 512x512 的图像
        device=device,
    )
    
    timesteps = torch.tensor([timestep], device=device)
    with torch.no_grad():
        unet_output = pipe.unet(
            latents,
            timesteps,
            encoder_hidden_states=text_embeddings
        ).sample
        
    return unet_output


img_feature = {}

with open('ccpd_data/CCPD_text_only.txt', 'r', encoding="utf-8") as f:
    for idx, line in enumerate(f.readlines()):
        poem = line.strip().split(',')[1]
        print(poem)
        img_feature[line.strip()] = get_unet_output(poem)


with open('ccpd_unet_feture_dict.pkl', 'wb') as f:
    pickle.dump(img_feature, f)